# The influences of the economists (influenced and have been influenced)

In [17]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import scipy.stats
import pandas as pd
from collections import Counter
from operator import itemgetter
import networkx as nx
%matplotlib inline


ModuleNotFoundError: No module named 'igraph'

In [3]:
query = """

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT DISTINCT ?economist ?name ?influencedBy ?influenced

WHERE { ?economist a dbo:Economist .
        ?economist foaf:name ?name .
        ?economist dbo:nationality ?nationality .
        ?economist owl:sameAs ?sameAs .
        ?economist dbp:birthDate ?Birth_Date .
        ?economist dbo:influencedBy ?influencedBy1 .
        BIND(STRAFTER(STR(?influencedBy1),"http://dbpedia.org/resource/") AS ?influencedBy2)
  BIND(REPLACE(STR(?influencedBy2),"[_]"," ") AS ?influencedBy3)
  BIND(REPLACE(STR(?influencedBy3),"[é]","e") AS ?influencedBy4)
  BIND(REPLACE(STR(?influencedBy4),"J. M. Keynes", "John Maynard Keynes") AS ?influencedBy)
        ?economist dbo:influenced ?influenced1 .
        BIND(STRAFTER(STR(?influenced1),"http://dbpedia.org/resource/") AS ?influenced2)
  BIND(REPLACE(STR(?influenced2),"[_]"," ") AS ?influenced3)
  BIND(REPLACE(STR(?influenced3),"[é]","e") AS ?influenced4)
  BIND(REPLACE(STR(?influenced4),"J. M. Keynes", "John Maynard Keynes") AS ?influenced)
        FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
        FILTER (str(?influenced) != "")
        FILTER (str(?influencedBy) != "")
}
"""

In [4]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")

In [5]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [6]:
rc = sparql.queryAndConvert()
type(rc)

dict

In [7]:
# Number of rows in the result
len(rc['results']['bindings'])

692

In [19]:
# Inspect the first three rows
i = 0
for l in rc['results']['bindings']:
    if i < 5:
        print(l)
        i += 1

{'economist': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ludwig_Lachmann'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': ''}, 'influencedBy': {'type': 'literal', 'value': 'Ludwig von Mises'}, 'influenced': {'type': 'literal', 'value': 'Don Lavoie'}}
{'economist': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ludwig_Lachmann'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Ludwig Lachmann'}, 'influencedBy': {'type': 'literal', 'value': 'Ludwig von Mises'}, 'influenced': {'type': 'literal', 'value': 'Don Lavoie'}}
{'economist': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ludwig_Lachmann'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': ''}, 'influencedBy': {'type': 'literal', 'value': 'Ludwig von Mises'}, 'influenced': {'type': 'literal', 'value': 'Bruce Caldwell (historian of economic thought)'}}
{'economist': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ludwig_Lachmann'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'va

In [35]:
# Extract the values from the first five rows
i = 0
results= []
for l in rc['results']['bindings']:
    if i < 10000 :
        try: 
            name = l['name']['value']
            result.append([l['economist']['value'],l['influencedBy']['value'], l['influenced']['value']])
        except :
            try: 
                name = l['prefLabel']['value']
                result.append([l['economist']['value'],l['influencedBy']['value'], l['influenced']['value']])
            except :
                result.append([l['economist']['value'],l['influencedBy']['value'], l['influenced']['value']])

        i += 1

In [36]:
df = pd.DataFrame(results, columns=['economist','influencedBy', 'influenced'])
print(df.index)

Index([], dtype='object')


In [34]:
MAdr=df.as_matrix()
print(MAdr)

[['http://dbpedia.org/resource/Ludwig_Lachmann' 'Ludwig von Mises'
  'Don Lavoie']
 ['http://dbpedia.org/resource/Ludwig_Lachmann' 'Ludwig von Mises'
  'Don Lavoie']
 ['http://dbpedia.org/resource/Ludwig_Lachmann' 'Ludwig von Mises'
  'Bruce Caldwell (historian of economic thought)']
 ...
 ['http://dbpedia.org/resource/Mark_Gertler_(economist)' 'Ben Bernanke'
  'Jordi Galí']
 ['http://dbpedia.org/resource/Wolfgang_Stützel' 'John Maynard Keynes'
  'Peter Bofinger']
 ['http://dbpedia.org/resource/Wolfgang_Stützel' 'John Maynard Keynes'
  'Heiner Flassbeck']]


/home/sylvain/anaconda3/envs/Github/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [37]:
net=nx.Graph()
for result in results['economist']:    
    for i in range(len(result['economist']['influenced'])):
        eco=result['economist']['influenced'][i]['influencedBy']
        if economist not in net:
            net.add_node(eco, rc=rc[economist], nEconomist=0)
for result in results['economist']:    
    for i in range(len(result['economist']['influenced'])-1):
        eco1=result['economist']['influenced'][i]['influencedBy']
        net.nodes[eco1]['nEconomists']+=1
        for j in range(i+1, len(result['economist']['influenced'])):
            eco2=result['economist']['influenced'][j]['influencedBy']
            net.nodes[eco2]['nEconomists']+=1
            net.add_edge(eco1,eco2)

TypeError: list indices must be integers or slices, not str